In [16]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from data_storage import create_connection
from stockstats import StockDataFrame

In [17]:
connection = create_connection("../database/crypto_billionairs.db")

In [18]:
table = "ETHUSDT_5m_complete_history_long"
table_short = "ETHUSDT_5m_complete_history_short"

In [19]:
df_long = pd.read_sql_query(f'select * from {table}', connection)
df_short = pd.read_sql_query(f'select * from {table_short}', connection)

In [20]:
df_backtesting = pd.DataFrame()
if "5m" not in table:
    df_backtesting["time"] = pd.to_datetime(df_long["open time"], unit='ms')
    df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_long[['open', 'high', 'low', 'close', 'volume']]
    df_backtesting = df_backtesting.set_index("time")

else:
    df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_long[['open', 'high', 'low', 'close', 'volume']]

In [21]:
# df_backtesting["time"] = pd.to_datetime(df_long["open time"], unit='ms')
# df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_long[['open', 'high', 'low', 'close', 'volume']]
# df_backtesting = df_backtesting.set_index("time")

In [22]:
def calculate_osc(data):
    
   
    df_temp = pd.DataFrame()    
    stock = StockDataFrame.retype(data)
    
    df_temp["stochastic_oscillator"] = stock.get('kdjk')
    
    return df_temp["stochastic_oscillator"]

def calucalte_rsi(data):
    
    df_temp = pd.DataFrame()    
    stock = StockDataFrame.retype(data)
    
    df_temp["relative_strength_index"] = stock.get("rsi_30")
    
    return df_temp["relative_strength_index"]

def momentum(df, lag):
    return df["close"].pct_change(periods=lag)

def init_close_long_signal(df):
    
    return df_long['close_indicator']

def init_buy_signal(df):
    
    return df_long['buy_indicator']

def init_short_signal(df):
    
    return df_short['short_signals']

def init_close_short_signal(df):
    
    return df_short['short_close_signals']
    

In [23]:
class OscRsi_5m_long_short(Strategy):
   
    lag = 4
    
    def init(self):
        # compute the rsi and stochastic oscillator with stockstats and return the buy signal of the current row
        
        self.osc = self.I(calculate_osc, self.data.df)
        self.rsi = self.I(calucalte_rsi, self.data.df)
        self.buy_init = self.I(init_buy_signal, self.data.df)
        self.close_buy_init = self.I(init_close_long_signal, self.data.df)
        self.short_init = self.I(init_short_signal, self.data.df)
        self.close_short_init = self.I(init_close_short_signal, self.data.df)
        self.data.df[['Open', 'High', 'Low', 'Close', "Volume",]] = self.data.df[['open', 'high', 'low', 'close', "volume"]]
        self.returns = self.I(momentum, self.data.df, self.lag)
        self.data.df.drop(self.data.df.columns.difference(['Open', 'High', 'Low', 'Close', "Volume", "kdjk", "rsi_30"]), 1, inplace=True)
        
       
    
    def next(self):
        
        if self.buy_init[-1] == 1 and self.position.is_long is False:
            self.position.close()
            self.buy()
            
        elif self.close_buy_init[-1] == 1 and self.position.is_long:
             self.position.close()
             
             
        elif self.short_init == 1 and self.position.is_short is False:
            self.position.close()
            self.sell()


In [24]:
bt = Backtest(df_backtesting, OscRsi_5m_long_short, cash=10_000, commission=.001)
stats = bt.run()
stats

<ipython-input-24-8d03f2706277>:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_backtesting, OscRsi_5m_long_short, cash=10_000, commission=.001)


Start                                       0
End                                    113865
Duration                               113865
Exposure Time [%]                     46.2851
Equity Final [$]                      73187.5
Equity Peak [$]                       74011.3
Return [%]                            631.875
Buy & Hold Return [%]                 131.947
Return (Ann.) [%]                           0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                                0
Max. Drawdown [%]                    -34.1636
Avg. Drawdown [%]                     -1.6885
Max. Drawdown Duration                  18244
Avg. Drawdown Duration                315.052
# Trades                                   22
Win Rate [%]                          81.8182
Best Trade [%]                        35.9844
Worst Trade [%]                      -19.5819
Avg. Trade [%]                    

In [25]:
bt.plot()

Row(id='3663', ...)

In [26]:
# %%time 
# stats = bt.optimize(lag=[12, 24, 48, 96, 120, 144, 168],
#                     sell_ = np.arange(60, 90, 3).tolist(),#sell_threshold = np.arange(0, -0.15, -0.01).tolist(),
#                     rsi_lower_bound = np.arange(10, 40, 3).tolist(),
#                     rsi_upper_bound = np.arange(12, 50, 3).tolist(),
#                     #osc_lower_bound = np.arange(10, 90, 5).tolist(),
#                     #osc_upper_bound = np.arange(10, 100, 5).tolist(),
#                     #maximize='Equity Final [$]')#,
#                     constraint= lambda param: param.rsi_lower_bound < param.rsi_upper_bound)
# stats

In [27]:
stats._strategy

<Strategy OscRsi_5m_long_short>

In [28]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,5,3583,5599,1875.81394,1843.26,-162.76970,-0.017355,3583,5599,2016
1,5,8176,10192,1766.13437,2042.80,1383.32815,0.156650,8176,10192,2016
2,5,11558,13574,2172.75058,2486.14,1566.94710,0.144236,11558,13574,2016
3,3,19660,21676,3673.13947,3901.89,686.25159,0.062277,19660,21676,2016
4,6,22803,22837,2008.68668,2731.50,4336.87992,0.359844,22803,22837,34
5,-6,22837,32122,2728.76850,2261.26,2805.05100,0.171326,22837,32122,9285
6,9,32122,34138,2263.52126,1820.28,-3989.17134,-0.195819,32122,34138,2016
7,8,34162,36178,1950.51857,2327.20,3013.45144,0.193119,34162,36178,2016
8,10,40857,42873,1862.91105,2298.26,4353.48950,0.233693,40857,42873,2016
9,9,44995,47011,2621.05844,3247.54,5638.33404,0.239019,44995,47011,2016


In [29]:
trades = pd.DataFrame(stats['_trades'])

In [30]:
trades.to_excel(f"trades_{table}_short.xlsx")